# Azure Cognitive Search Vector Search Code Sample with Azure OpenAI and Langchain
This code demonstrates how to use Azure Cognitive Search with OpenAI and Langchain for generating embeddings and performing similarity searches.

## Prerequisites 

To run the code, install the following packages:

In [ ]:
! pip install ../build/azure_search_documents-11.4.0b4-py3-none-any.whl 
! pip install ../build/acslangchain-0.0.4-py3-none-any.whl 
! pip install openai
! pip install python-dotenv
! pip install numpy
! pip install pydantic
! pip install langchain 

## Import required libraries

In [14]:
import os, json
import openai
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import BaseRetriever
from acslangchain.azuresearch import AzureSearch

## Configure OpenAI settings
Configure the OpenAI settings to use Azure OpenAI or OpenAI

In [15]:
# Load environment variables from a .env file using load_dotenv():
load_dotenv()

openai.api_type = "azure"
openai.api_base = os.getenv('OPENAI_ENDPOINT')
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")
model: str = os.getenv('OPENAI_EMBEDDING_DEPLOYED_MODEL', "text-embedding-ada-002")

## Configure vector store settings
 
Set up the vector store settings using environment variables:

In [16]:
vector_store_address: str = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT')
vector_store_password: str = os.getenv("AZURE_SEARCH_ADMIN_KEY")
index_name: str = os.getenv("AZURE_SEARCH_INDEX_NAME", "langchain-vector-demo")

## Create embeddings and vector store instances
 
Create instances of the OpenAIEmbeddings and AzureSearch classes:

In [17]:
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(model=model, chunk_size=1)  
vector_store: AzureSearch = AzureSearch(azure_cognitive_search_name=vector_store_address,  
                                        azure_cognitive_search_key=vector_store_password,  
                                        index_name=index_name,  
                                        embedding_function=embeddings.embed_query)  


## Insert text and embeddings into vector store
 
Add texts and metadata from the JSON data to the vector store:

In [18]:
# Read the text-sample.json
with open('../data/text-sample.json', 'r', encoding='utf-8') as file:
    input_data = json.load(file)

max_input: int = 100
vector_store.add_texts(
    texts=list(map(lambda x: x['content'], input_data[0:max_input])),
    metadatas=list(map(lambda x: { 'title': x['title'], 'key': x['id'], 'tag': x['category'] }, input_data[0:max_input])),
    keys=list(map(lambda x: x['id'], input_data[0:max_input]))
)

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100']

## Perform a vector similarity search
 
Execute a pure vector similarity search using the similarity_search() method:

In [19]:
# Perform a similarity search
vector_store.similarity_search(query="tools for software development", k=3)

[Document(page_content='Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.', metadata={'title': 'Azure DevOps', 'key': '9', 'tag': 'Developer Tools'}),
 Document(page_content='Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports various platforms, such 

In [24]:
# Perform a similarity search multi-lingual (Italian)
vector_store.similarity_search(query="strumenti per sviluppo software", k=3)

[Document(page_content='Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.', metadata={'title': 'Azure DevOps', 'key': '9', 'tag': 'Developer Tools'}),
 Document(page_content='Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports various platforms, such 

## Perform a Hybrid Search

Execute an hybrid search using the hybrid_search() method:

In [21]:
# Perform a hybrid search 
vector_store.hybrid_search(query="scalable storage solution", k=3)

[Document(page_content='Azure Storage is a scalable, durable, and highly available cloud storage service that supports a variety of data types, including blobs, files, queues, and tables. It provides a massively scalable object store for unstructured data. Storage supports data redundancy and geo-replication, ensuring high durability and availability. It offers a variety of data access and management options, including REST APIs, SDKs, and Azure Portal. You can secure your data using encryption at rest and in transit.', metadata={'title': 'Azure Storage', 'key': '4', 'tag': 'Storage'}),
 Document(page_content='Azure Table Storage is a fully managed, NoSQL datastore that enables you to store and query large amounts of structured, non-relational data. It provides features like automatic scaling, schema-less design, and a RESTful API. Table Storage supports various data types, such as strings, numbers, and booleans. You can use Azure Table Storage to store and manage your data, build scal

## Perform a Hybrid Search with a filter

Execute an hybrid search using the hybrid_search() method with filters:

In [22]:
# Perform a hybrid search with a filter
vector_store.hybrid_search(query="What's Azure Cognitive Search?", k=3, filters="tag eq 'AI + Machine Learning'")

[Document(page_content='Azure Cognitive Search is a fully managed search-as-a-service that enables you to build rich search experiences for your applications. It provides features like full-text search, faceted navigation, and filters. Azure Cognitive Search supports various data sources, such as Azure SQL Database, Azure Blob Storage, and Azure Cosmos DB. You can use Azure Cognitive Search to index your data, create custom scoring profiles, and integrate with other Azure services. It also integrates with other Azure services, such as Azure Cognitive Services and Azure Machine Learning.', metadata={'title': 'Azure Cognitive Search', 'key': '40', 'tag': 'AI + Machine Learning'}),
 Document(page_content='Azure Cognitive Services are a set of AI services that enable you to build intelligent applications with powerful algorithms using just a few lines of code. These services cover a wide range of capabilities, including vision, speech, language, knowledge, and search. They are designed to 

## Perform a Semantic Hybrid Search
Execute a semantic hybrid search with the semantic_hybrid_search() method:

In [23]:
# Perform a Semantic Hybrid Search
vector_store.semantic_hybrid_search(query="what is azure search?", k=3)

[Document(page_content='Azure Cognitive Search is a fully managed search-as-a-service that enables you to build rich search experiences for your applications. It provides features like full-text search, faceted navigation, and filters. Azure Cognitive Search supports various data sources, such as Azure SQL Database, Azure Blob Storage, and Azure Cosmos DB. You can use Azure Cognitive Search to index your data, create custom scoring profiles, and integrate with other Azure services. It also integrates with other Azure services, such as Azure Cognitive Services and Azure Machine Learning.', metadata={'title': 'Azure Cognitive Search', 'key': '40', 'tag': 'AI + Machine Learning', 'captions': {'text': 'Azure Cognitive Search is a fully managed search-as-a-service that enables you to build rich search experiences for your applications. It provides features like full-text search, faceted navigation, and filters. Azure Cognitive Search supports various data sources, such as Azure SQL Database